In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import pandas as pd
import requests
import json
import time
import timeit
import os
import bar_chart_race as bcr

In [ ]:
## compile driver championship points in a dictionary, with drivers' name as key and the championships scores as values, stored in a list

In [3]:
gp_len = 19 # change this according to how many races are already finished

statDriver = {}
dri_lst = []

first = True

for i in range(gp_len):
    url = f'http://ergast.com/api/f1/2021/{i+1}/results.json'
    
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    
    resp_json = response.json()

    for j in range(20):
        dri = resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['Driver']['code']
        try:
            statDriver[dri].append(resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['points'])
        except KeyError:
            statDriver[dri] = []
            statDriver[dri].append(resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['points'])
        
print(statDriver)

{'HAM': ['25', '19', '25', '25', '7', '0', '18', '19', '12', '27', '18', '7.5', '19', '0', '25', '10', '19', '18'], 'VER': ['18', '25', '18', '19', '25', '0', '26', '25', '26', '3', '2', '12.5', '25', '2', '18', '18', '25', '25'], 'BOT': ['16', '0', '16', '15', '0', '0', '12', '15', '18', '16', '0', '0', '15', '18', '10', '26', '8', '0'], 'NOR': ['12', '15', '10', '4', '15', '10', '10', '10', '15', '12', '0', '0', '1', '18', '7', '6', '4', '1'], 'PER': ['10', '0', '12', '10', '12', '25', '15', '12', '8', '0', '0', '0', '4', '10', '2', '15', '15', '15'], 'LEC': ['8', '12', '8', '12', '0', '12', '0', '6', '4', '18', '0', '2', '10', '12', '0', '12', '12', '10'], 'RIC': ['6', '8', '2', '8', '0', '2', '8', '0', '6', '10', '0', '6', '0', '27', '12', '0', '10', '0'], 'SAI': ['4', '10', '0', '6', '18', '4', '0', '8', '10', '8', '15', '0.5', '6', '8', '15', '4', '6', '8'], 'TSU': ['2', '0', '0', '0', '0', '6', '0', '1', '0', '1', '8', '0', '0', '0', '0', '0', '2', '0'], 'STR': ['1', '4', '0', '

In [ ]:
## this section is does the same as above expressed only as functions to compare runtimes

In [ ]:
## this is just a demonstration of ways of executing loops

In [26]:
# using for loops

def naive(gp_len):
    statDriver = {}
    dri_lst = []

    first = True

    for i in range(gp_len):
        url = f'http://ergast.com/api/f1/2021/{i+1}/results.json'

        payload={}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)

        resp_json = response.json()

        for j in range(20):
            dri = resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['Driver']['code']
            # replace Kubica with Raikonnen
            if dri == 'KUB':
                dri = 'RAI'
            try:
                statDriver[dri].append(resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['points'])
            except KeyError:
                statDriver[dri] = []
                statDriver[dri].append(resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['points'])
    return statDriver

In [ ]:
## using recursion

In [3]:
def rec_json(j, resp_json, len_resp, statDriver):
    if j == len_resp:
        return statDriver
    else:
        dri = resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['Driver']['code']
        # replace Kubica with Raikonnen
        if dri == 'KUB':
            dri = 'RAI'
        try:
            statDriver[dri].append(float(resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['points']))
            j += 1
        except KeyError:
            statDriver[dri] = []
            statDriver[dri].append(float(resp_json['MRData']['RaceTable']['Races'][0]['Results'][j]['points']))
            j += 1
            
        return rec_json(j, resp_json, len_resp, statDriver)

In [4]:
def rec_stat(i, gp_len, output):
    if i == gp_len:
        return output
    else:
        url = f'http://ergast.com/api/f1/2021/{i+1}/results.json'

        payload={}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)
        resp_json = response.json()
        len_resp = len(resp_json['MRData']['RaceTable']['Races'][0]['Results'])
        
        output = rec_json(0, resp_json, len_resp, output)
        i+=1
        
        return rec_stat(i, gp_len, output)

In [ ]:
## this section is just evaluating run time of the two looping algorithm
## uncomment if you want to re-run

In [8]:
# time_naive = []

# for i in range(10):
#     start = time.time()
#     naive(18)
#     time_naive.append(time.time()-start)

In [9]:
# time_rec = []

# for i in range(10):
#     start = time.time()
#     rec_stat(0, 18, {})
#     time_rec.append(time.time()-start)

In [10]:
# assert(naive(18) == rec_stat(0, 18, {}))

In [12]:
# import statistics as stat

# print(f'{"naive_runtime:"}{stat.mean(time_naive):.2f}{" seconds"}')
# print(f'{"recursive_runtime:"}{stat.mean(time_rec):.2f}{" seconds"}')

naive_runtime:16.19 seconds
recursive_runtime:16.21 seconds


In [5]:
gp_dict = rec_stat(0, 18, {})

In [6]:
df = pd.DataFrame(gp_dict)

In [7]:
df

,HAM,VER,BOT,NOR,PER,LEC,RIC,SAI,TSU,STR,RAI,GIO,OCO,RUS,VET,MSC,GAS,LAT,ALO,MAZ
0,25.0,18.0,16.0,12.0,10.0,8.0,6.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19.0,25.0,0.0,15.0,0.0,12.0,8.0,10.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0
2,25.0,18.0,16.0,10.0,12.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0
3,25.0,19.0,15.0,4.0,10.0,12.0,8.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,7.0,25.0,0.0,15.0,12.0,0.0,0.0,18.0,0.0,4.0,0.0,1.0,2.0,0.0,10.0,0.0,8.0,0.0,0.0,0.0
5,0.0,0.0,0.0,10.0,25.0,12.0,2.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,18.0,0.0,15.0,0.0,8.0,0.0
6,18.0,26.0,12.0,10.0,15.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0,4.0,0.0
7,19.0,25.0,15.0,10.0,12.0,6.0,0.0,8.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
8,12.0,26.0,18.0,15.0,8.0,4.0,6.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0
9,27.0,3.0,16.0,12.0,0.0,18.0,10.0,8.0,1.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0


In [8]:
dfT = df.transpose()

In [9]:
columns = ['Bahrain ','Imola ','Portugal ','Spain ','Monaco ','Azerbaijan ','France ','Styria ','Austria ','Britain ','Hungary ','Belgium ','Netherlands ','Italy ','Russia ','Turkey ','USA', 'Mexico']

In [10]:
columns_stripped = [s.strip() for s in columns] # remove trailing spaces on each list value

In [11]:
dfT.columns = columns_stripped

In [12]:
dfT

,Bahrain,Imola,Portugal,Spain,Monaco,Azerbaijan,France,Styria,Austria,Britain,Hungary,Belgium,Netherlands,Italy,Russia,Turkey,USA,Mexico
HAM,25.0,19.0,25.0,25.0,7.0,0.0,18.0,19.0,12.0,27.0,18.0,7.5,19.0,0.0,25.0,10.0,19.0,18.0
VER,18.0,25.0,18.0,19.0,25.0,0.0,26.0,25.0,26.0,3.0,2.0,12.5,25.0,2.0,18.0,18.0,25.0,25.0
BOT,16.0,0.0,16.0,15.0,0.0,0.0,12.0,15.0,18.0,16.0,0.0,0.0,15.0,18.0,10.0,26.0,8.0,0.0
NOR,12.0,15.0,10.0,4.0,15.0,10.0,10.0,10.0,15.0,12.0,0.0,0.0,1.0,18.0,7.0,6.0,4.0,1.0
PER,10.0,0.0,12.0,10.0,12.0,25.0,15.0,12.0,8.0,0.0,0.0,0.0,4.0,10.0,2.0,15.0,15.0,15.0
LEC,8.0,12.0,8.0,12.0,0.0,12.0,0.0,6.0,4.0,18.0,0.0,2.0,10.0,12.0,0.0,12.0,12.0,10.0
RIC,6.0,8.0,2.0,8.0,0.0,2.0,8.0,0.0,6.0,10.0,0.0,6.0,0.0,27.0,12.0,0.0,10.0,0.0
SAI,4.0,10.0,0.0,6.0,18.0,4.0,0.0,8.0,10.0,8.0,15.0,0.5,6.0,8.0,15.0,4.0,6.0,8.0
TSU,2.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
STR,1.0,4.0,0.0,0.0,4.0,0.0,1.0,4.0,0.0,4.0,0.0,0.0,0.0,6.0,0.0,2.0,0.0,0.0


In [ ]:
# perform cummulative sum on columns

In [13]:
dfT_sum = dfT.cumsum(axis=1)

In [14]:
dfT_sum

,Bahrain,Imola,Portugal,Spain,Monaco,Azerbaijan,France,Styria,Austria,Britain,Hungary,Belgium,Netherlands,Italy,Russia,Turkey,USA,Mexico
HAM,25.0,44.0,69.0,94.0,101.0,101.0,119.0,138.0,150.0,177.0,195.0,202.5,221.5,221.5,246.5,256.5,275.5,293.5
VER,18.0,43.0,61.0,80.0,105.0,105.0,131.0,156.0,182.0,185.0,187.0,199.5,224.5,226.5,244.5,262.5,287.5,312.5
BOT,16.0,16.0,32.0,47.0,47.0,47.0,59.0,74.0,92.0,108.0,108.0,108.0,123.0,141.0,151.0,177.0,185.0,185.0
NOR,12.0,27.0,37.0,41.0,56.0,66.0,76.0,86.0,101.0,113.0,113.0,113.0,114.0,132.0,139.0,145.0,149.0,150.0
PER,10.0,10.0,22.0,32.0,44.0,69.0,84.0,96.0,104.0,104.0,104.0,104.0,108.0,118.0,120.0,135.0,150.0,165.0
LEC,8.0,20.0,28.0,40.0,40.0,52.0,52.0,58.0,62.0,80.0,80.0,82.0,92.0,104.0,104.0,116.0,128.0,138.0
RIC,6.0,14.0,16.0,24.0,24.0,26.0,34.0,34.0,40.0,50.0,50.0,56.0,56.0,83.0,95.0,95.0,105.0,105.0
SAI,4.0,14.0,14.0,20.0,38.0,42.0,42.0,50.0,60.0,68.0,83.0,83.5,89.5,97.5,112.5,116.5,122.5,130.5
TSU,2.0,2.0,2.0,2.0,2.0,8.0,8.0,9.0,9.0,10.0,18.0,18.0,18.0,18.0,18.0,18.0,20.0,20.0
STR,1.0,5.0,5.0,5.0,9.0,9.0,10.0,14.0,14.0,18.0,18.0,18.0,18.0,24.0,24.0,26.0,26.0,26.0


In [ ]:
# transpose the dataframe to properly orient rows and column values as desired on the output of the bar chart race

In [15]:
dfT_sumT = dfT_sum.transpose()

In [16]:
dfT_sumT

,HAM,VER,BOT,NOR,PER,LEC,RIC,SAI,TSU,STR,RAI,GIO,OCO,RUS,VET,MSC,GAS,LAT,ALO,MAZ
Bahrain,25.0,18.0,16.0,12.0,10.0,8.0,6.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Imola,44.0,43.0,16.0,27.0,10.0,20.0,14.0,14.0,2.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0
Portugal,69.0,61.0,32.0,37.0,22.0,28.0,16.0,14.0,2.0,5.0,0.0,0.0,8.0,0.0,0.0,0.0,7.0,0.0,5.0,0.0
Spain,94.0,80.0,47.0,41.0,32.0,40.0,24.0,20.0,2.0,5.0,0.0,0.0,10.0,0.0,0.0,0.0,8.0,0.0,5.0,0.0
Monaco,101.0,105.0,47.0,56.0,44.0,40.0,24.0,38.0,2.0,9.0,0.0,1.0,12.0,0.0,10.0,0.0,16.0,0.0,5.0,0.0
Azerbaijan,101.0,105.0,47.0,66.0,69.0,52.0,26.0,42.0,8.0,9.0,1.0,1.0,12.0,0.0,28.0,0.0,31.0,0.0,13.0,0.0
France,119.0,131.0,59.0,76.0,84.0,52.0,34.0,42.0,8.0,10.0,1.0,1.0,12.0,0.0,30.0,0.0,37.0,0.0,17.0,0.0
Styria,138.0,156.0,74.0,86.0,96.0,58.0,34.0,50.0,9.0,14.0,1.0,1.0,12.0,0.0,30.0,0.0,37.0,0.0,19.0,0.0
Austria,150.0,182.0,92.0,101.0,104.0,62.0,40.0,60.0,9.0,14.0,1.0,1.0,12.0,0.0,30.0,0.0,39.0,0.0,20.0,0.0
Britain,177.0,185.0,108.0,113.0,104.0,80.0,50.0,68.0,10.0,18.0,1.0,1.0,14.0,0.0,30.0,0.0,39.0,0.0,26.0,0.0


In [ ]:
## download the drivers' headshots and contructors' cars

In [17]:
url = "https://fantasy-api.formula1.com/partner_games/f1/players"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

resp_json = response.json()

In [18]:
position = resp_json['players'][0]['headshot']['profile']
Photo_URL = resp_json['players'][0]['headshot']['profile']
Photo_name = resp_json['players'][0]['last_name'][:3].upper() # only used the 1st 3 letters to use a filename

In [95]:
for i in range(30):
    position = resp_json['players'][i]['position']
    Photo_URL = resp_json['players'][i]['headshot']['profile']
    
    if position == 'Driver':
        Photo_name = resp_json['players'][i]['last_name'][:3].upper()
        Picture_request = requests.get(Photo_URL)
        if Picture_request.status_code == 200:
            with open(os.path.join('images/Drivers', f'{Photo_name}{".png"}'), 'wb') as f:
                f.write(Picture_request.content)
    else:
        Photo_name = resp_json['players'][i]['team_abbreviation']
        Picture_request = requests.get(Photo_URL)
        if Picture_request.status_code == 200:
            with open(os.path.join('images/Constructor', f'{Photo_name}{".png"}'), 'wb') as f:
                f.write(Picture_request.content)

In [26]:
bcr.bar_chart_race(
    df = dfT_sumT,
    filename = 'f1_240.mp4',
    img_label_folder = 'images/Drivers',
    steps_per_period=30,
    interpolate_period=False,
    fig_kwargs={
        'dpi': 240,
        'facecolor': '#F8FAFF'
    },
    n_bars=10,
    fixed_max=False,
    period_length=900,
    title={
        'label': "F1 2021 Season World Driver's Championship",
        'weight': 'bold',
    }
)